# YOLOv3 Training

# Package
- numpy = 1.15.2
- opencv-python =3.4.3
- tqdm = 4.27.0
- matplotlib = 2.2.3
- tensorflow = 1.10.0
- keras = 2.2.2

# 架構流程 
- 1. 動所需參數train_annot_folde,train_image_folder,cache_name,labels,saved_weights_name
- 2. 執行訓練

In [1]:
#! /usr/bin/env python
# https://github.com/experiencor/keras-yolo3
import argparse
import os
import numpy as np
import json
from voc import parse_voc_annotation
from yolo import create_yolov3_model, dummy_loss
from generator import BatchGenerator
from utils.utils import normalize, evaluate, makedirs
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
from callbacks import CustomModelCheckpoint, CustomTensorBoard
from utils.multi_gpu_model import multi_gpu_model
import tensorflow as tf
import keras
from keras.models import load_model

Using TensorFlow backend.
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:
def create_training_instances(
    train_annot_folder,
    train_image_folder,
    train_cache,
    valid_annot_folder,
    valid_image_folder,
    valid_cache,
    labels,
):
    # parse annotations of the training set
    train_ints, train_labels = parse_voc_annotation(train_annot_folder, train_image_folder, train_cache, labels)
    # parse annotations of the validation set, if any, otherwise split the training set
    if os.path.exists(valid_annot_folder):
        valid_ints, valid_labels = parse_voc_annotation(valid_annot_folder, valid_image_folder, valid_cache, labels)
    else:
        print("valid_annot_folder not exists. Spliting the trainining set.")

        train_valid_split = int(0.8*len(train_ints))
        np.random.seed(0)
        np.random.shuffle(train_ints)
        np.random.seed()

        valid_ints = train_ints[train_valid_split:]
        train_ints = train_ints[:train_valid_split]

    # compare the seen labels with the given labels
    if len(labels) > 0:
        overlap_labels = set(labels).intersection(set(train_labels.keys()))

        print('Seen labels: \t'  + str(train_labels) + '\n')
        print('Given labels: \t' + str(labels))

        # return None, None, None if some given label is not in the dataset
        if len(overlap_labels) < len(labels):
            print('Some labels have no annotations! Please revise the list of labels')
            return None, None, None
    else:
        print('No labels are provided. Train on all seen labels.')
        print(train_labels)
        labels = train_labels.keys()

    max_box_per_image = max([len(inst['object']) for inst in (train_ints + valid_ints)])

    return train_ints, valid_ints, sorted(labels), max_box_per_image

In [3]:
def create_callbacks(saved_weights_name, model_to_save):
    
    early_stop = EarlyStopping(
        monitor     = 'loss', 
        min_delta   = 0.01, 
        patience    = 5, 
        mode        = 'min', 
        verbose     = 1
    )
    checkpoint = CustomModelCheckpoint(
        model_to_save   = model_to_save,
        filepath        = saved_weights_name,# + '{epoch:02d}.h5', 
        monitor         = 'loss', 
        verbose         = 1, 
        save_best_only  = True, 
        mode            = 'min', 
        period          = 1
    )
    reduce_on_plateau = ReduceLROnPlateau(
        monitor  = 'loss',
        factor   = 0.1,
        patience = 2,
        verbose  = 1,
        mode     = 'min',
        epsilon  = 0.01,
        cooldown = 0,
        min_lr   = 0
    )    
    return [early_stop, checkpoint, reduce_on_plateau]

In [4]:
def create_model(
    nb_class, 
    anchors, 
    max_box_per_image, 
    max_grid, batch_size, 
    warmup_batches, 
    ignore_thresh,  
    saved_weights_name, 
    lr,
    grid_scales,
    obj_scale=5,
    noobj_scale=1,
    xywh_scale=1,
    class_scale=1  
):
    train_model, infer_model = create_yolov3_model(
        nb_class            = nb_class, 
        anchors             = anchors, 
        max_box_per_image   = max_box_per_image, 
        max_grid            = max_grid, 
        batch_size          = batch_size, 
        warmup_batches      = warmup_batches,
        ignore_thresh       = ignore_thresh,
        grid_scales         = grid_scales,
        obj_scale           = obj_scale,
        noobj_scale         = noobj_scale,
        xywh_scale          = xywh_scale,
        class_scale         = class_scale
    )  

    # load the pretrained weight if exists, otherwise load the backend weight only
    if os.path.exists(saved_weights_name): 
        print("\nLoading pretrained weights.\n")
        train_model.load_weights(saved_weights_name)
    else:
        train_model.load_weights("backend.h5", by_name=True)          

    optimizer = Adam(lr=lr, clipnorm=0.001)
    train_model.compile(loss=dummy_loss, optimizer=optimizer)             

    return train_model, infer_model

# 參數設置

In [5]:
#更改訓練資料夾路徑，以及cache_name
# Parse the annotations

min_input_size = 288
max_input_size = 448
# Create the generators 
anchors = [55,69, 75,234, 133,240, 136,129, 142,363, 203,290, 228,184, 285,359, 341,260]
labels = ['CNV', 'DME', 'DRUSEN']

# 可在這更改batch_size
batch_size = 2

train_annot_folder = './retina/annots/'
train_image_folder = './retina/images/'
cache_name = 'retina.pkl'

#更改儲存model名稱
# Create the model 
saved_weights_name = 'retina.h5'

warmup_epochs = 3
nb_epochs = 30
train_times = 8
ignore_thresh = 0.5
learning_rate = 1e-4
grid_scales = [1,1,1]
obj_scale = 5

valid_annot_folder = ''
valid_image_folder = ''
valid_cache_name = ''

In [6]:
train_ints, valid_ints, labels, max_box_per_image = create_training_instances(
    train_annot_folder,
    train_image_folder,
    cache_name,
    valid_annot_folder,
    valid_image_folder,
    valid_cache_name,
    labels
)
print('\nTraining on: \t' + str(labels) + '\n')


train_generator = BatchGenerator(
    instances           = train_ints, 
    anchors             = anchors,   
    labels              = labels,        
    downsample          = 32, # ratio between network input's size and network output's size, 32 for YOLOv3
    max_box_per_image   = max_box_per_image,
    batch_size          = batch_size,
    min_net_size        = min_input_size,
    max_net_size        = max_input_size,   
    shuffle             = True, 
    jitter              = 0.3, 
    norm                = normalize
)

valid_generator = BatchGenerator(
    instances           = valid_ints, 
    anchors             = anchors,   
    labels              = labels,        
    downsample          = 32, # ratio between network input's size and network output's size, 32 for YOLOv3
    max_box_per_image   = max_box_per_image,
    batch_size          = batch_size,
    min_net_size        = min_input_size,
    max_net_size        = max_input_size,   
    shuffle             = True, 
    jitter              = 0.0, 
    norm                = normalize
)


if os.path.exists(saved_weights_name): 
    warmup_epochs = 0
warmup_batches = warmup_epochs * (train_times*len(train_generator))

train_model, infer_model = create_model(
    nb_class            = len(labels), 
    anchors             = anchors, 
    max_box_per_image   = max_box_per_image, 
    max_grid            = [max_input_size, max_input_size], 
    batch_size          = batch_size, 
    warmup_batches      = warmup_batches,
    ignore_thresh       = 0.5,
    saved_weights_name  = saved_weights_name,
    lr                  = 1e-4,
    grid_scales         = [1,1,1],

)

# Start training
callbacks = create_callbacks(saved_weights_name, infer_model)

train_model.fit_generator(
    generator        = train_generator, 
    steps_per_epoch  = len(train_generator) * train_times, 
    epochs           = nb_epochs + warmup_epochs, 
    verbose          = 2,
    callbacks        = callbacks, 
    workers          = 4,
    max_queue_size   = 8
)




valid_annot_folder not exists. Spliting the trainining set.
Seen labels: 	{'CNV': 252, 'DME': 146, 'DRUSEN': 183}

Given labels: 	['CNV', 'DME', 'DRUSEN']

Training on: 	['CNV', 'DME', 'DRUSEN']

Instructions for updating:
Use `tf.cast` instead.

Instructions for updating:
Use tf.print instead of tf.Print. Note that tf.print returns a no-output operator that directly prints the output. Outside of defuns or eager mode, this operator will not be executed unless it is directly specified in session.run or used as a control dependency for other operators. This is only a concern in graph mode. Below is an example of how to ensure tf.print executes in graph mode:
```python
    sess = tf.compat.v1.Session()
    with sess.as_default():
        tensor = tf.range(10)
        print_op = tf.print(tensor)
        with tf.control_dependencies([print_op]):
          out = tf.add(tensor, tensor)
        sess.run(out)
    ```
Additionally, to use tf.print in python 2.7, users must make sure to import
th

/opt/conda/lib/python3.7/site-packages/keras/callbacks/callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '





Epoch 1/30
resizing:  352 352
resizing:  448 448
resizing:  320 320
resizing:  384 384
resizing:  352 352
resizing:  288 288
resizing:  384 384
resizing:  448 448
resizing:  352 352
resizing:  384 384
resizing:  448 448
resizing:  416 416
resizing:  448 448
resizing:  448 448
resizing:  416 416
resizing:  416 416
resizing:  448 448
resizing:  384 384
resizing:  416 416
resizing:  288 288
resizing:  448 448
resizing:  416 416
resizing:  352 352
resizing:  320 320
resizing:  448 448
resizing:  448 448
resizing:  352 352
resizing:  384 384
resizing:  448 448
resizing:  352 352
resizing:  320 320
resizing:  320 320
resizing:  320 320
resizing:  288 288
resizing:  384 384
resizing:  384 384
resizing:  320 320
resizing:  352 352
resizing:  448 448
resizing:  448 448
resizing:  416 416
resizing:  448 448
resizing:  416 416
resizing:  448 448
resizing:  320 320
resizing:  352 352
resizing:  448 448
resizing:  288 288
resizing:  384 384
resizing:  288 288
resizing:  384 384
resizing:  448 44

In [7]:
import argparse
import os
import numpy as np
import json
from voc import parse_voc_annotation
from yolo import create_yolov3_model
from generator import BatchGenerator
from utils.utils import normalize, evaluate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras.models import load_model


config_path = './config.json'

with open(config_path) as config_buffer:    
    config = json.loads(config_buffer.read())

###############################
#   Create the validation generator
###############################  
valid_ints, labels = parse_voc_annotation(
    config['valid']['valid_annot_folder'], 
    config['valid']['valid_image_folder'], 
    config['valid']['cache_name'],
    config['model']['labels']
)

labels = labels.keys() if len(config['model']['labels']) == 0 else config['model']['labels']
labels = sorted(labels)

valid_generator = BatchGenerator(
    instances           = valid_ints, 
    anchors             = config['model']['anchors'],   
    labels              = labels,        
    downsample          = 32, # ratio between network input's size and network output's size, 32 for YOLOv3
    max_box_per_image   = 0,
    batch_size          = config['train']['batch_size'],
    min_net_size        = config['model']['min_input_size'],
    max_net_size        = config['model']['max_input_size'],   
    shuffle             = True, 
    jitter              = 0.0, 
    norm                = normalize
)

###############################
#   Load the model and do evaluation
###############################
os.environ['CUDA_VISIBLE_DEVICES'] = config['train']['gpus']

infer_model = load_model(config['train']['saved_weights_name'])

# compute mAP for all the classes
average_precisions = evaluate(infer_model, valid_generator)

# print the score
for label, average_precision in average_precisions.items():
    print(labels[label] + ': {:.4f}'.format(average_precision))
print('mAP: {:.4f}'.format(sum(average_precisions.values()) / len(average_precisions)))             

/opt/conda/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


CNV: 0.9020
DME: 0.9540
DRUSEN: 0.9279
mAP: 0.9280
